# ROB2004 Final Project - pick and place objects

The goal of this project is to solve a simple manipulation task: picking up objects and moving them in a bowl.

## Instructions
* Answer all questions in the notebook
* You will need to submit on Brightspace: 
    1. the code you wrote to answer the questions in a Jupyter Notebook. The code should be runnable as is.
    2. a 2-3 pages report in pdf format (pdf only) detailing the methodology you followed to answer the questions as well as answers to the questions that require a typeset answer. You may add the plots in the report (does not count for the page limit) or in the Jupyter notebook.
* This is an individual project

We will use a model of the Frank-Emika Panda robot. This robot has 7 revolute joints and its kinematics is described in the picture below.

You will need to install pybullet on your computer. Please reach out as soon as possible for help if you do not know how to do it.

<div>
<img src="./panda.jpg" width="200"/>
</div>

# Question

Implement a controller using the code skeletong below in order to get the robot to go and pick up the red blocks and drop them in the green bowl. The bowl position is $(-0.3,0.55,0.65)$ (in spatial frame coordinates) and the blocks positions are $(0.35,0.58,0.65)$ and $(0.15,0.67,0.65)$ respectively.

<div>
<img src="./manipulation_scene.jpg" width="300"/>
</div>

You are free to use the methods that you want to solve the task, with the following constraints:
* You cannot use any external library apart from numpy and scipy
* You need to use at least one controller in the end-effector space
* You need to generate smooth motions
* You need to compensate for the gravity of the robot

In your report, describe the controller and justify your choices. Analyze the behavior of the system (include plots of the end-effector trajectories, velocities, joint trajectories, etc as you see fit).

In [1]:
import numpy as np
import time

import matplotlib.pyplot as plt

from panda import Simulator, PandaRobot

np.set_printoptions(precision=4, suppress = True)

# Helper functions

We provide a set of helper functions (forward kinematics, Jacobians, gravity terms) that can be used to implement the desired controllers. These functions are implemented in the PandaRobot class (in the panda.py file).
Examples are shown below:

In [2]:
# create an instance of the helper class
my_robot = PandaRobot()

# compute the forward kinematics for an arbitrary joint configuration
q = np.random.uniform(-1,1,7)
pose = my_robot.FK(np.array([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,0.0]))
print(f'the pose of the end-effector for joint configuration\n\n {q}\n\n is\n\n {pose}')

the pose of the end-effector for joint configuration

 [-0.9906 -0.1687 -0.6908 -0.904  -0.417   0.3023 -0.188 ]

 is

 [[-0.7071  0.7071  0.      0.    ]
 [ 0.7071  0.7071 -0.      0.5545]
 [-0.      0.     -1.      0.6245]
 [ 0.      0.      0.      1.    ]]


In [3]:
# we can also get the Jacobian in various frames 
# it is sufficient to pass the name of the frame as an argument as shown below

q = np.random.uniform(-1,1,7)

spatial_jacobian = my_robot.get_jacobian(q, 'S')
body_jacobian = my_robot.get_jacobian(q, 'B')

# we can even get the Jacobian in a frame place at the same position as the end-effector 
# frame but oriented like the spatial frame
oriented_jacobian = my_robot.get_jacobian(q, 'O')

print(f'for joint configuration \n\n {q} \n \n the spatial Jacobian is \n\n {spatial_jacobian}\n\n')
print(f'the body Jacobian is \n\n {body_jacobian}\n\n')
print(f'and the Jacobian in a frame placed at the same position as the end-effector but oriented like the spatial frame is\n\n {oriented_jacobian}')

for joint configuration 

 [-0.4448 -0.8101 -0.7444  0.7805  0.5313 -0.7372 -0.6183] 
 
 the spatial Jacobian is 

 [[ 0.     -0.9027 -0.3117  0.463  -0.8053  0.5868  0.5393]
 [ 0.      0.4303 -0.6538 -0.7381 -0.5816 -0.8096  0.3774]
 [ 1.      0.      0.6895 -0.4907  0.115   0.0145 -0.7528]
 [ 0.     -0.1433  0.2177  0.5337  0.2855  0.454   0.0262]
 [ 0.     -0.3006 -0.1038  0.2606 -0.4147  0.3385  0.099 ]
 [ 0.      0.     -0.      0.1115 -0.0974  0.5271  0.0684]]


the body Jacobian is 

 [[ 0.0951  0.8073 -0.3563 -0.9125 -0.1118 -0.9861 -0.    ]
 [ 0.6513 -0.493   0.0312 -0.2265 -0.6628  0.1663  0.    ]
 [-0.7528 -0.3245 -0.9338  0.3406 -0.7404  0.      1.    ]
 [-0.3734  0.1606 -0.1404  0.1065 -0.0067  0.0178 -0.    ]
 [ 0.1336  0.376  -0.0313 -0.1167  0.0011  0.1055  0.    ]
 [ 0.0684 -0.1718  0.0525  0.2077 -0.     -0.088  -0.    ]]


and the Jacobian in a frame placed at the same position as the end-effector but oriented like the spatial frame is

 [[ 0.     -0.9027 -0.3117  0.

In [4]:
# finally it is possible to compute the effect of gravity on the joints
# this can for example be used to do gravity compensation when controlling the robot

q = np.random.uniform(-1,1,7)

g = my_robot.g(q)

print(f'for joint configuration \n\n {q} \n \n the gravity force seen on the joints is \n \n {g}')

for joint configuration 

 [ 0.5813 -0.5937  0.5674 -0.2706  0.621   0.169  -0.929 ] 
 
 the gravity force seen on the joints is 
 
 [ 0.     25.6201 -3.001  -8.9814 -0.4112  1.0082  0.    ]


In [5]:
def linear_compute_trajectory(position_init, position_goal, t_init, t_goal, t):
    desired_position = 0
    desired_velocity = 0
    
    # we return the answer
    T = t_goal - t_init
    desired_position = position_init + (t-t_init)/T*(position_goal-position_init)
    desired_velocity = (1/T)*(position_goal-position_init)
    return desired_position, desired_velocity

# Code Skeleton for the simulation

Feel free to change the `run_time` variable to match your needs. The class `Simulator` contains all the code for the simulation. The code skeleton currently implement a simple PD controller in joint space that just keeps a desired current position.

Importantly you can control both the joints and the gripper of the robot:
* `simulator.send_joint_torque(joint_torques)` sends a vector of dimension 7 setting the torques of the joints
* `simulator.gripper_move(gripper_position)` sends a vector of dimension 2 setting the position of the fingers of the gripper. The fingers can have positions from 0 to 0.04cm (you can fully close the gripper by setting `simulator.gripper_move([0,0])` and fully open it by setting `simulator.gripper_move([0.04,0.04)`)

In [6]:
# here we create a simulation and reset the state of the robot
simulator = Simulator()
simulator.reset_state([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,0.0])

# we create a robot object so we can use its helper functions
my_robot = PandaRobot()

# duration of the simulation 
run_time = 40. # 40 seconds

# simulation time step
dt = 0.005
num_steps = int(run_time/dt)


# we store information
ndofs = 7 # number of degrees of freedom (excluding the gripper)
measured_positions = np.zeros([num_steps,ndofs])
measured_velocities = np.zeros_like(measured_positions)
desired_torques = np.zeros_like(measured_positions)
desired_positions = np.zeros_like(measured_positions)
desired_velocities = np.zeros_like(measured_positions)
time = np.zeros([num_steps])

# target positions for the blocks and bowl
block_pos1 = np.array([0.15, .65, 0.8])  # position of block 1 in spatial frame, y and z coordinates adjusted
block_pos2 = np.array([0.35, .6, 0.8])  # position of block 2 in spatial frame, y and z coordinates adjusted
bowl_pos = np.array([-0.3, .55, 1])  # position of the bowl in spatial frame, set z = 1 to be above the bowl

detour =  np.array([0.35, .6, .95]) # small detour for smooth movements between bowl to 2nd block
targets = [block_pos1, bowl_pos, detour, block_pos2, bowl_pos] # BLOCK 1 -> BOWL -> ABOVE BLOCK 2 -> BLOCK 2 -> BOWL

time_to_goal = 10.0 # seconds for each movement
# the PD gains
P = np.array([100., 100., 100., 100., 100., 100., 50.])
D = np.array([10.,10,10,10,10,10,1.])


### this is the main control loop
for i in range(num_steps):
    # get the current time and save it
    time[i] = dt * i
    
    # we get the position and velocities of the joints
    q, dq = simulator.get_state()
    measured_positions[i,:] = q
    measured_velocities[i,:] = dq
    
    ## controller: TODO HERE IMPLEMENT YOUR CONTROLLER TO SOLVE THE TASK
    ## you will need to replace the naive PD controller implemented below
    
    if time[i] == 0: # First Target is the first block
        initial_pos = np.array(q) # initial position is starting position
        target_pos = targets[0] # target position is first block
        
        # final_desired = simulator.get_IK(target_pos)[:7]
        final_desired = np.array([-0.22177322062709837, 0.49468037379071483, -0.005746305404099892,
                         -1.7211763409326888, 0.0034140305483085593, 2.2158487097731796, -2.5850778164490613]) # joint angles
        
        offset = 0 # offset time
        time_to_goal = 10 # time to reach destination
        
    elif time[i] == 10: # Second Target is the bowl
        initial_pos = final_desired # initial position now the first block
        target_pos = targets[1] # New target position is above the bowl
        
        # final_desired = simulator.get_IK(target_pos)[:7]
        # new joint angles
        final_desired = np.array([0.3202109923062064, 0.21232344510144538, 0.20032442966538067,
                                  -1.6197976328007184, -0.043369670475953075, 1.8277717464226517, -1.8290133524675016]) 
        
        offset = 10 # update offset time
        time_to_goal = 10 # time to reach destination
        
    elif time[i] == 20: # Third Target is the above the second block (need to take a small detour)
        initial_pos = final_desired # initial position now the bowl
        target_pos = targets[2] # New target position is above the 2nd block
        
        # final_desired = simulator.get_IK(target_pos)[:7]
        # new joint angles
        final_desired = np.array([-0.3415470369586015, 0.4716007016299897, -0.24413533780068575,
                                    -1.3770924546679233, 0.11400851786807825, 1.8349774666324055, -2.946005193580198])
        
        offset = 20  # update offset time
        time_to_goal = 5 # time to reach destination (5 seconds)
        
    elif time[i] == 25: # Fourth Target is finally the second block
        initial_pos = final_desired # initial position now above the 2nd block
        target_pos = targets[3] # New target position is the 2nd block
        
        # final_desired = simulator.get_IK(target_pos)[:7]
        # New joint angles
        final_desired = np.array([-0.34906284308726543, 0.5894546554198747, -0.21758886090339905,
                                 -1.5819075217989076, 0.14442202472059404, 2.1555618126397453, -2.967116606067746])
        
        offset = 25 # update offset time
        time_to_goal = 5 # time to reach destination (5 seconds; 10 seconds in total b/w bowl to 2nd block)
        
    elif time[i] == 30: # Drop last block off at bowl
        initial_pos = final_desired # initial position now the 2nd block
        target_pos = targets[4] # Last new target position is bowl again
        
        # final_desired = simulator.get_IK(target_pos)[:7]
        # New joint angles
        final_desired = np.array([0.3938860958906943, 0.20984903035733937, 0.1178294589174742,
                                 -1.6198697762777292, -0.025325530935699083, 1.8282301820531681, -1.8405917610570857])
        
        offset = 30 # update offset time
        time_to_goal = 10 # time to reach destination
        
    elif time[i] == 40: # Simulation END
        final_desired = np.zeros(ndofs) # set all joint angles to 0 to stop movement
    
    # desired joint positions and velocities
    # print(final_desired)
    
    # Linear Velocity
    q_des, dq_des = linear_compute_trajectory(initial_pos, final_desired, offset, offset + time_to_goal, time[i])
    

    # we save the desired positions/velocities for later plotting 
    desired_positions[i,:] = q_des
    desired_velocities[i,:] = dq_des
    
    ##PD controller
    error = q_des - q # the position error for all the joints
    d_error = dq_des-dq # the velocity error for all the joints
    
    
    # we compute the desired torques as a PD controller
    # compensate for gravity
    joint_torques = np.diag(P) @ error + np.diag(D) @ d_error + my_robot.g(q)
    
    # we start by closing the gripper and then open it after 5 seconds
    gripper_position = [0,0] # close
    
    if 8.0 <= time[i] <= 10.0: # For two seconds...
        gripper_position = [0.04, 0.04] # open gripper to pick up first block
    
    elif 19.0 <= time[i] <= 21.0: # For two seconds...
        gripper_position = [0.04, 0.04] # open gripper to drop the first block into bowl
    
    elif 28.0 <= time[i] <= 30.0: # For two seconds...
        gripper_position = [0.04, 0.04] # open gripper to grab the 2nd block 
        
    elif 39.0 <= time[i]: # After 39 seconds...
        gripper_position = [0.04, 0.04] # keep gripper open (drop off the 2nd block into bowl too)
    
    # we send them to the robot and do one simulation step
    simulator.send_joint_torque(joint_torques)
    simulator.gripper_move(gripper_position)
    simulator.step()
